In [4]:
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.errors import HttpError
from dotenv import load_dotenv
import csv
import os
import pandas as pd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import base64
import pickle

# Retrieve variables from .env
SERVICE_ACCOUNT_FILE = os.getenv('SERVICE_ACCOUNT_FILE')
SPREADSHEET_ID = os.getenv('SPREADSHEET_ID')
print(SERVICE_ACCOUNT_FILE)
SAMPLE_RANGE_NAME = 'Sheet1'  # Update the range if necessary
# The scopes required by the Sheets and Drive APIs
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# Load environment variables
load_dotenv()

testing-for-ecocar-ff6b5e5bbe13.json


True

In [5]:
# Replace this with the path to your actual CSV file
csv_file_path = 'headcount.csv'

# Initialize a dictionary to store sub-teams and their headcounts
team_headcounts = {}

# Open the CSV file and read it
with open(csv_file_path, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    next(csvreader, None)  # Skip the header row
    
    for row in csvreader:
        # Assuming the sub-team is in the first column and the headcount is in the second
        sub_team = row[0]
        headcount = int(row[1])  # Convert headcount to integer

        # Store the sub-team and headcount in the dictionary
        team_headcounts[sub_team] = headcount

In [6]:
def get_google_sheet(service_account_file, spreadsheet_id, range_name, fill_value='N/A'):
    """Retrieve sheet data using OAuth credentials and Google Python API, 
    filling missing values with a specified fill_value."""
    print(service_account_file)
    creds = Credentials.from_service_account_file(service_account_file)
    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    values = result.get('values', [])

    if not values:
        print("No data found.")
        return []

    # Ensure all rows have the same number of columns
    num_columns = len(values[0])  # Number of columns is based on the first row (header row)
    for row in values:
        while len(row) < num_columns:
            row.append(fill_value)  # Append fill_value to rows with fewer elements than headers

    return values
values = get_google_sheet(SERVICE_ACCOUNT_FILE, SPREADSHEET_ID, SAMPLE_RANGE_NAME)
df = pd.DataFrame(values[1:], columns=values[0])  # Using the first row as column names

new_df = df[df['Team Assigned'] == "N/A"]

testing-for-ecocar-ff6b5e5bbe13.json


In [7]:
df

,Timestamp,Email Address,First Name,Last Name,Team Preference 1,Team Preference 2,Team Preference 3,Team Assigned,Accept
0,2/17/2024 14:58:05,ham.konsing@gmail.com,Walbeg,Stuart,PCM - 1,EiM - 2,Cavs - 1,PCM - 1,N/A
1,2/17/2024 14:59:32,ham.konsing@gmail.com,Edward,John,SDI -2,EiM - 3,Cavs - 2,Cavs - 1,N/A
2,2/17/2024 14:59:35,manyuk_9@hotmail.com,Hen,Hao,SDI -2,PCM - 1,Cavs - 2,N/A,N/A
3,2/17/2024 14:59:35,manyuk_9@hotmail.com,Peter,Noah,SDI -2,Cavs - 1,Cavs - 2,EiM - 2,N/A


In [8]:
new_df

,Timestamp,Email Address,First Name,Last Name,Team Preference 1,Team Preference 2,Team Preference 3,Team Assigned,Accept
2,2/17/2024 14:59:35,manyuk_9@hotmail.com,Hen,Hao,SDI -2,PCM - 1,Cavs - 2,N/A,N/A


In [9]:
def assignMembers(new_df, team_headcounts, csv_file_path):
    unassigned_members = {}
    assigned_df = pd.DataFrame([], columns=values[0])
    index = 0

    for i in range(1, 4):
        for idx, row in new_df.iterrows():
            team = row[f'Team Preference {i}']
            if team_headcounts[team] > 0 and row["Team Assigned"] == "N/A":
                assigned_df = pd.concat([assigned_df, pd.DataFrame([row])], ignore_index=True)
                assigned_df.at[index, 'Team Assigned'] = team
                new_df = new_df.drop(idx)
                team_headcounts[team] -= 1
                index += 1
    
    # Update the CSV file with new headcounts
    with open(csv_file_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Sub-team', 'headcount'])  # Assuming these are your column headers
        for team, count in team_headcounts.items():
            csvwriter.writerow([team, count])

    for idx, row in new_df.iterrows():
        unassigned_members[row["Email Address"]] = f"{row['First Name']} {row['Last Name']}"

    return assigned_df, unassigned_members

In [10]:
unassigned_members = {}
assigned_members_df, unassigned_members = assignMembers(new_df, team_headcounts, csv_file_path)

In [11]:
print(f"{'Email Address':<25} {'Name'}")
for key, val in unassigned_members.items():
    print(f"{key:<25} {val}")

Email Address             Name
manyuk_9@hotmail.com      Hen Hao


In [12]:
assigned_members_df

,Timestamp,Email Address,First Name,Last Name,Team Preference 1,Team Preference 2,Team Preference 3,Team Assigned,Accept


In [13]:
def write_team_assignments_to_sheet(assigned_members_df, df, service_account_file, spreadsheet_id):
    """Write team assignments to the Google Sheet based on assigned members dataframe."""
    creds = Credentials.from_service_account_file(service_account_file, scopes=['https://www.googleapis.com/auth/spreadsheets'])
    service = build('sheets', 'v4', credentials=creds)

    # No need to fetch current values from the sheet as we are using the index from df
    update_data = []
    for _, assigned_row in assigned_members_df.iterrows():
        email_address = assigned_row['Email Address']
        team_assigned = assigned_row['Team Assigned']
        
        # Find the index of the row in df that matches the assigned member's email address
        index_in_df = df.index[df['Email Address'] == email_address].tolist()[0] + 2  # +2 to adjust for Google Sheets indexing and header
        
        update_data.append({
            'range': f'H{index_in_df}',  # Writing to column H
            'values': [[team_assigned]]
        })

    # Batch update the values
    body = {
        'valueInputOption': 'USER_ENTERED',
        'data': update_data
    }
    service.spreadsheets().values().batchUpdate(spreadsheetId=spreadsheet_id, body=body).execute()

In [14]:
write_team_assignments_to_sheet(assigned_members_df, df, SERVICE_ACCOUNT_FILE, SPREADSHEET_ID)

In [15]:
# Functions used to send emails

# Email sending function
def send_email(service, user_id, message):
    try:
        message = service.users().messages().send(userId=user_id, body=message).execute()
        print(f"Email sent to {message['id']}")
    except HttpError as error:
        print(f'An error occurred: {error}')

# Create email message
def create_message(sender, to, subject, message_text):
    message = MIMEMultipart()
    message['from'] = sender
    message['to'] = to
    message['subject'] = subject
    message.attach(MIMEText(message_text, 'plain'))
    raw_msg = base64.urlsafe_b64encode(message.as_bytes()).decode()
    return {'raw': raw_msg}

# Initialize Gmail API
def get_gmail_service(client_secret_file, scopes):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, scopes)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return build('gmail', 'v1', credentials=creds)

In [16]:
# Map of team names to team leader emails
team_leader_emails = {
    "PCM - 1": "ham.konsing@gmail.com",
    "Cavs - 1": "manyuk_9@hotmail.com",
    "EiM - 2": "welbaghdadi@ucdavis.edu"
}

# Replace 'client_secret_....json' with your actual client secret file name
client_secret_file = 'client_secret_171250538774-1hv52jle9rmtc3hpkfhttgs7sdr0ts8d.apps.googleusercontent.com.json'
sender_email = "ham.konsing@gmail.com"  # The email you're sending from

In [17]:
# Code to send email to team leaders informing them of new members

# Function to notify team leaders
def notify_team_leaders(df, team_leader_emails, client_secret_file, sender_email):
    scopes = ['https://www.googleapis.com/auth/gmail.send']
    service = get_gmail_service(client_secret_file, scopes)
    
    for index, row in df.iterrows():
        team_assigned = row['Team Assigned']
        if team_assigned in team_leader_emails:
            team_leader_email = team_leader_emails[team_assigned]
            subject = "New Team Member"
            message_text = f"Dear Team Leader,\n\nI'm pleased to inform you that: {row['First Name']} {row['Last Name']} ({row['Email Address']}) has joined our team, ({row['Team Assigned']}).\n\nPlease update the team roster to include this new addition. Thank you for your attention to this matter.\n\nBest regards,\nAutomated System\nEcoCAR"
            message = create_message(sender_email, team_leader_email, subject, message_text)
            send_email(service, 'me', message)

# Call the function with your dataframe 'df'
notify_team_leaders(df, team_leader_emails, client_secret_file, sender_email)

Email sent to 18e1b66c95a14814
Email sent to 18e1b66c94aaab4f
Email sent to 18e1b66cbb63951a


In [18]:
# Code to congratulate the student on getting into the team

from google.oauth2.credentials import Credentials

def congratulate_new_member(df, client_secret_file, sender_email):
    scopes = ['https://www.googleapis.com/auth/gmail.send']
    service = get_gmail_service(client_secret_file, scopes)
    
    for index, row in df.iterrows():
        first_name = row['First Name']
        last_name = row['Last Name']
        email_address = row['Email Address']
        team_name = row['Team Assigned']
        
        subject = "Welcome to the Team!"
        message_text = f"Dear {first_name} {last_name},\n\nCongratulations on joining our team, {team_name}! We are thrilled to have you with us and look forward to achieving great things together.\n\nPlease remember to check our team calendar for upcoming meetings and events. Staying informed will help us all stay on track and make the most out of our collaboration.\n\nWelcome aboard!\n\nBest regards,\nKonsing Ham Lopez\nEco Car Team Leader\n{sender_email}"
        message = create_message(sender_email, email_address, subject, message_text)
        send_email(service, 'me', message)

# Send email to student
congratulate_new_member(assigned_members_df, client_secret_file, sender_email)

In [19]:
# Code to send email to unassigned members, (tell them, they're on waitlist)

def notify_unassigned_members(unassigned_members, client_secret_file, sender_email):
    """
    Notifies individuals that they are placed on the wait list due to no open slots on their preferred teams.

    Parameters:
    - unassigned_members: Dictionary with email addresses as keys and names as values.
    - client_secret_file: Path to the client secret file for Gmail API authentication.
    - sender_email: Email address from which the notification will be sent.
    """
    scopes = ['https://www.googleapis.com/auth/gmail.send']
    service = get_gmail_service(client_secret_file, scopes)
    
    for email, name in unassigned_members.items():
        subject = "Update on Team Assignment"
        message_text = f"Dear {name},\n\nWe have received your preferences for team assignments, and we wanted to provide you with an update. Unfortunately, due to a high level of interest and limited availability, we are unable to place you on any of your preferred teams at this time.\n\nWe understand this may be disappointing, but you have been placed on our wait list. Should a slot open up on one of your preferred teams, we will reach out to you immediately.\n\nWe appreciate your understanding and patience. If you have any questions or wish to discuss other ways you can contribute in the meantime, please do not hesitate to contact us.\n\nBest regards,\nEcoCAR Automated System"
        message = create_message(sender_email, email, subject, message_text)
        send_email(service, 'me', message)

# Call the function with your unassigned members dictionary
notify_unassigned_members(unassigned_members, client_secret_file, sender_email)


Email sent to 18e1b66ccb816259
